In [1]:
"https://www.che168.com/nanjing/a0_0msdgscncgpi1lto1csp3exx0/?pvareaid=102179"

#一级页面的正则'<li class="cards-li list-photo-li ".*?<a href="(.*?)".*?</li>'

#二级页面的'<div class="car-box">.*?<h3 class="car-brand-name">(.*?)</h3>.*?<h4>(.*?)</h4>.*?<h4>(.*?)</h4>.*?<h4>(.*?)</h4>.*?<h4>(.*?)</h4>.*?<span class="price" id="overlayPrice">¥(.*?)<b>.*?</div>'

'https://www.che168.com/nanjing/a0_0msdgscncgpi1lto1csp3exx0/?pvareaid=102179'

In [22]:
"""
汽车之家数据抓取
"""
from urllib import request
import re
import time
import random

class CarSpider:
    def __init__(self):
        self.url="https://www.che168.com/nanjing/a0_0msdgscncgpi1lto1csp{}exx0/?pvareaid=102179"
        self.headers={
            "User-Agent":"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/14.0.835.163 Safari/535.1"
        }
        self.i=0#计数的变量

    def get_html(self,url):
        """功能函数1：获取响应内容"""
        req=request.Request(url=url,headers=self.headers)
        res=request.urlopen(req)
        html=res.read().decode("gb2312","ignore")  #看一下网页的编码

        return html

    def re_func(self,regex,html):  #regex是正则表达式的参数
        """ 功能函数2：解析提取数据"""
        pattern=re.compile(regex,re.S)
        r_list=pattern.findall(html)
        
        return r_list
    
    def parse_html(self,one_url):
        """数据抓取函数---从一级开始"""
        one_html=self.get_html(one_url)
        one_regex='<li class="cards-li list-photo-li ".*?<a href="(.*?)".*?</li>'
        href_list=self.re_func(one_regex,one_html) #提取除了每一个info的二级连接["...","..."]里面是路径
        #https://www.che168.com/
        for href in href_list:
            car_url="https://www.che168.com/"+href
            self.get_data(car_url)#抓取一辆车的数据
            #抓取一次歇一会
            time.sleep(random.uniform(3,4))#生成指定范围的浮点数

    def get_data(self,car_url):
        """功能是抓取一辆车的详情数据"""
        two_html=self.get_html(car_url)
        two_regex='<div class="car-box">.*?<h3 class="car-brand-name">(.*?)</h3>.*?<h4>(.*?)</h4>.*?<h4>(.*?)</h4>.*?<h4>(.*?)</h4>.*?<h4>(.*?)</h4>.*?<span class="price" id="overlayPrice">&#165(.*?)<b>.*?</div>'
        car_list=self.re_func(two_regex,two_html)
        # car_list[("夏利","2万公里"),]
        item={}
        item["name"]=car_list[0][0].strip()
        item["km"]=car_list[0][1].strip()
        item["time"]=car_list[0][2].strip()
        item["type"]=car_list[0][3].split("/")[0].strip()
        item["displace"]=car_list[0][3].split("/")[1].strip()
        item["city"]=car_list[0][4].strip()
        item["price"]=car_list[0][5].strip()
        print(item)
    
    def run(self):
        """程序入口"""
        for i in range(1,3):
            
            url=self.url.format(i)
            self.parse_html(url)
    
if __name__ =="__main__":
    spider=CarSpider()
    spider.run()








{'name': '夏利 2008款 N3+ 1.0L 三厢助力 国IV', 'km': '6.8万公里', 'time': '2009年12月', 'type': '手动', 'displace': '1L', 'city': '南京', 'price': ';0.68'}
{'name': '同悦 2010款 1.3L 手动豪华型', 'km': '9万公里', 'time': '2011年03月', 'type': '手动', 'displace': '1.3L', 'city': '南京', 'price': ';0.70'}


KeyboardInterrupt: 

In [29]:
#使用md5给url加个密

"""
汽车之家数据抓取，利用mysql增量爬虫
"""
from urllib import request
import re
import time
import random

import pymysql
from hashlib import md5

import sys

class CarSpider:
    def __init__(self):
        self.url="https://www.che168.com/nanjing/a0_0msdgscncgpi1lto1csp{}exx0/?pvareaid=102179"
        self.headers={
            "User-Agent":"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/14.0.835.163 Safari/535.1"
        }
        self.i=0#计数的变量
        #数据库
        self.db=pymysql.connect("localhost","root","243698","douban",charset="utf8")
        self.cursor=self.db.cursor()

    def get_html(self,url):
        """功能函数1：获取响应内容"""
        req=request.Request(url=url,headers=self.headers)
        res=request.urlopen(req)
        html=res.read().decode("gb2312","ignore")  #看一下网页的编码

        return html

    def re_func(self,regex,html):  #regex是正则表达式的参数
        """ 功能函数2：解析提取数据"""
        pattern=re.compile(regex,re.S)
        r_list=pattern.findall(html)
        
        return r_list
    
    def md5_url(self,url):
        """对url地址进行md5加密"""
        s=md5()
        s.update(url.encode())
        return s.hexdigest()

    def parse_html(self,one_url):
        """数据抓取函数---从一级开始"""
        one_html=self.get_html(one_url)
        one_regex='<li class="cards-li list-photo-li ".*?<a href="(.*?)".*?</li>'
        href_list=self.re_func(one_regex,one_html) #提取除了每一个info的二级连接["...","..."]里面是路径
        #https://www.che168.com/
        for href in href_list:
            car_url="https://www.che168.com/"+href
            url_md5=self.md5_url(car_url)
            sel="select * from request_finger where finger=%s"
            self.cursor.execute(sel,[url_md5])
            result=self.cursor.fetchall()
            #result两种情况，空或者非空
            # 如果之前没抓过再去抓：
            if not result:
                self.get_data(car_url)#抓取一辆车的数据
                #抓取一次歇一会
                time.sleep(random.uniform(5,6))#生成指定范围的浮点数
                #抓取之后把指纹存入数据库
                ins="insert into request_finger values(%s)"
                self.cursor.execute(ins,[url_md5])
                self.db.commit()
            else:
                #sys.exit("抓取完成")  #结束进程 ，因为一般新闻网站，按照顺序发部的，如果不是，这边就调整一下
                continue

    def get_data(self,car_url):
        """功能是抓取一辆车的详情数据"""
        two_html=self.get_html(car_url)
        two_regex='<div class="car-box">.*?<h3 class="car-brand-name">(.*?)</h3>.*?<h4>(.*?)</h4>.*?<h4>(.*?)</h4>.*?<h4>(.*?)</h4>.*?<h4>(.*?)</h4>.*?<span class="price" id="overlayPrice">&#165(.*?)<b>.*?</div>'
        car_list=self.re_func(two_regex,two_html)
        # car_list[("夏利","2万公里"),]
        item={}
        maxcount=0
        while maxcount<3:
            try:
                item["name"]=car_list[0][0].strip()
                item["km"]=car_list[0][1].strip()
                item["time"]=car_list[0][2].strip()
                item["type"]=car_list[0][3].split("/")[0].strip()
                item["displace"]=car_list[0][3].split("/")[1].strip()
                item["city"]=car_list[0][4].strip()
                item["price"]=car_list[0][5].strip()
                ins="insert into cartab values(%s,%s,%s,%s,%s,%s,%s)"
                # 把上面的搞成列表
                li=[
                    item["name"],
                    item["km"],
                    item["time"],
                    item["type"],
                    item["displace"],
                    item["city"],
                    item["price"],
                ]
                self.cursor.execute(ins,li)
                self.db.commit()
                print(item)
                break
            except IndexError as e:
                time.sleep(2)
            maxcount+=1
            
         
    
    def run(self):
        """程序入口"""
        for i in range(1,3):
            url=self.url.format(i)
            self.parse_html(url)
        #断开数据库
        self.cursor.close()
        self.db.close()
    
if __name__ =="__main__":
    spider=CarSpider()
    spider.run()








{'name': '迷迪 2011款 宜商 1.3L 升级版标准型', 'km': '15万公里', 'time': '2012年06月', 'type': '手动', 'displace': '1.3L', 'city': '南京', 'price': ';1.28'}
{'name': '赛拉图 2006款 1.6L AT GLS', 'km': '17万公里', 'time': '2006年03月', 'type': '自动', 'displace': '1.6L', 'city': '南京', 'price': ';1.36'}
{'name': '乐风 2010款 1.4MT风度版', 'km': '10万公里', 'time': '2010年05月', 'type': '手动', 'displace': '1.4L', 'city': '南京', 'price': ';1.38'}
{'name': '福克斯 2009款 两厢 1.8L 手动舒适型', 'km': '10万公里', 'time': '2008年04月', 'type': '手动', 'displace': '1.8L', 'city': '南京', 'price': ';1.38'}
{'name': '名爵7 2008款 1.8T 手动豪华版', 'km': '12万公里', 'time': '2010年04月', 'type': '手动', 'displace': '1.8L', 'city': '南京', 'price': ';1.38'}
{'name': '世嘉 2009款 三厢 1.6L 手动舒适型', 'km': '10万公里', 'time': '2010年01月', 'type': '手动', 'displace': '1.6L', 'city': '南京', 'price': ';1.38'}
{'name': '爱丽舍 2005款 1.6SX 16V 手动挡', 'km': '9万公里', 'time': '2008年06月', 'type': '手动', 'displace': '1.6L', 'city': '南京', 'price': ';1.38'}
{'name': '标致206 2008款 1.6L 自动炫动版', 'km': '8万公里', 'time

URLError: <urlopen error [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。>

In [1]:
#requests请求模块
# 封装的很好，方便
import requests
url="http://www.baidu.com"
header={"zzz":"xxx"}
res=requests.get(url=url,headers=header)  #相当于req=requst.Request(url=url,headers=header) res=request.urlopen(req)

#获取响应内容
html=res.text




# 写法二
html=requests.get(url=url,headers=header).text



In [ ]:
#使用xpath来匹配 二手车
#链接
#//li[@class="cards-li list-photo-li"]/a[1]/@href

#name：//div[@class="car-box"]/h3/text()
#km: //div[@class="car-box"]//li[1]/h4/text()
#://div[@class="car-box"]//li[2]/h4/text()

#价格：//span[@class="price"]/text()

